In [ ]:
import io, os, sys
sys.setrecursionlimit(10**6)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
def bfs(C, F, l, src, snk) :
  V = len(C)
  Q = collections.deque()
  Q.append(src)
  l[src] = 0
  while Q :
    u = Q.popleft()
    for v in range(V) :
      if l[v] == -1 and C[u][v] - F[u][v] > 0 :
        l[v] = l[u] + 1
        Q.append(v)
  return l[snk] != -1

def dfs(C, F, W, l, u, snk, cur = 1 << 63) :
  if u == snk : return cur
  V = len(C)
  while W[u] < V :
    v = W[u]
    if l[v] == l[u] + 1 and C[u][v] > F[u][v] :
      A = min(cur, C[u][v] - F[u][v])
      tmp = dfs(C, F, W, l, v, snk, A)
      if tmp > 0 :
        F[u][v] += tmp
        F[v][u] -= tmp
        return tmp
    W[u] += 1
  return 0

def dinic(C, src, snk) :
  V = len(C)
  F = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    l = [-1] * V
    W = [0] * V
    if not bfs(C, F, l, src, snk) : break
    while True:
      tmp = dfs(C, F, W, l, src, snk)
      if tmp == 0 : break
      ret += tmp
  return ret, F

def sol() :
  N, M, K = map(int, input().split())
  V = N + M + 2
  C = [[0] * V for _ in range(V)]

  for i in range(N) :
    u = i + 1
    _, *l = map(int, input().split())
    C[0][u] = 1
    for j in l :
      v = N+j
      C[u][v] = 1

  for i in range(M) : #각각의 일을 담당하는 사람은 1명이어야 한다
    C[N+i+1][-1] = 1

  ans, F = dinic(C, 0, V-1)
  

sol()

- 이분매칭이 들어가야 시간제한도 풀리고 아직 모르는 dinic도 쓰지 않고 풀 수 있을 것 같다. 조금 나중에 풀자

5 5 1
5 1 2 3 4 5
1 1
1 1
1 1
2 1 5
;;
5 5 2
5 1 2 3 4 5
1 1
1 1
1 1
2 1 5
;;
5 5 3
5 1 2 3 4 5
1 1
1 1
1 1
2 1 5
;;
5 5 5
5 1 2 3 4 5
1 1
1 1
1 1
2 1 5
;;
3 6 3
3 1 2 3
1 3
4 3 4 5 6
;;
2 4 1
2 1 2
2 3 4

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**6)

input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
class Dinic:
  def __init__(self, n):
    self.lvl = [0] * n
    self.ptr = [0] * n
    self.q = [0] * n
    self.adj = [[] for _ in range(n)]

  def add_edge(self, a, b, c, rcap=0):
    self.adj[a].append([b, len(self.adj[b]), c, 0])
    self.adj[b].append([a, len(self.adj[a]) - 1, rcap, 0])

  def dfs(self, v, t, f):
    if v == t or not f:
      return f

    for i in range(self.ptr[v], len(self.adj[v])):
      e = self.adj[v][i]
      if self.lvl[e[0]] == self.lvl[v] + 1:
        p = self.dfs(e[0], t, min(f, e[2] - e[3]))
        if p:
          self.adj[v][i][3] += p
          self.adj[e[0]][e[1]][3] -= p
          return p
      self.ptr[v] += 1

    return 0

  def calc(self, s, t):
    flow, self.q[0] = 0, s
    for l in range(31):
      while True:
        self.lvl, self.ptr = [0] * len(self.q), [0] * len(self.q)
        qi, qe, self.lvl[s] = 0, 1, 1
        while qi < qe and not self.lvl[t]:
          v = self.q[qi]
          qi += 1
          for e in self.adj[v]:
            if not self.lvl[e[0]] and (e[2] - e[3]) >> (30 - l):
              self.q[qe] = e[0]
              qe += 1
              self.lvl[e[0]] = self.lvl[v] + 1

        p = self.dfs(s, t, INF)
        while p:
          flow += p
          p = self.dfs(s, t, INF)

        if not self.lvl[t]:
          break

    return flow

def sol() :
  N, P = map(int, input().split())
  C = Dinic(N)
  for _ in range(P) :
    a, b = map(lambda x : int(x) - 1, input().split())
    C.add_edge(a, b, 1)

  print(C.calc(0, 1))

def sol() :
  N, M, K = map(int, input().split())
  V = N + M + 3
  C = Dinic(V)

  C.add_edge(0, 1, K) #sink에서 bridge까지의 용량을 K로 설정해준다
  for i in range(N) :
    u = i + 2
    _, *l = map(int, input().split())
    C.add_edge(0, u, 1)
    C.add_edge(1, u, K) #bridge를 연결해준다
    for j in l :
      v = N+j+1
      C.add_edge(u, v, 1)

  for i in range(M) : #각각의 일을 담당하는 사람은 1명이어야 한다
    C.add_edge(N+i+2, V-1, 1)

  print(C.calc(0, V-1))

sol()

### 풀이
- 각 직원은 적어도 1개의 일은 할 수 있기 때문에 이분매칭의 그래프 연결처럼 1의 용량을 가진 간선을 이어주되,\
벌점을 처리하기 위한 추가 노드(bridge)를 구현해야 한다.
  - sink에서 bridge로 `K`의 용량을 가진 간선을 잇는다.
  - 각 직원이 몇개의 일을 하는지는 정해지지 않았으므로 bridge에서 각 직원에게 `K`용량을 가진 간선을 잇는다.
- 정해는 아니지만 최적화를 통해 더 빠르게 풀거나, 이분매칭으로도 풀 수 있다. 단순한 플로우로는 Dinic없이는 TLE가 났다.

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**6)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def bipartite(G, N, M) :
  def dfs(x) :
    if vis[x]: return False

    vis[x] = True
    for y in G[x]:
      if sel[y] == -1 :
        sel[y] = x
        return True
      
    for y in G[x] :
      if dfs(sel[y]):
        sel[y] = x
        return True
    return False

  sel = [-1] * M
  res = 0
  for i in range(N) :
    vis = [False] * N
    res += dfs(i)
  return res, sel

def sol() :
  N, M, K = map(int, input().split())
  G = [[] for _ in range(N)]

  U = set() #실제 직원들이 모두 할 수 있는 일
  for i in range(N) :
    _, *l = map(int, input().split())
    G[i] = [x-1 for x in l]
    U |= set(l)

  res, sel = bipartite(G, N, M) 
  print(min(len(U), res + K)) #각 직원이 1개의 일을 한뒤 벌점만큼 추가로 일을 할 수 있지만, 실제 직원이 가능한 일들의 양보단 적어야한다.

sol()